# 1 - Preparando ambiente

## Instalar requisitos

In [ ]:
%pip install -r requirements.txt

## Importar biblioteca

In [ ]:
from main import eContabilSite, insistir, AlwaysDoubleCheck

## Definir senha

Para que o processo possa ser executado corretamente, é preciso ter um arquivo config.json com o domínio, usuário e senha do ambiente informados para que a plataforma possa ser acessada.

Use o arquivo ```config_sample.json```para se basear ao criar o arquivo ```config.json``` corretamente.

# 2 - Iniciando a exportação

In [ ]:
print('Define o Log de erros como falso. (exibe erros e status no console)')
dbg = False
print('Carrega a classe, abre o navegador, loga na plataforma.')
es = eContabilSite(dbg)

print('Cria tabelas de acompanhamento do processo no banco de dados')
es.create_tables()

In [ ]:
print('Raspar dados de Clientes')
es.get_clients()

print('Raspar dados dos eventos')
es.get_movfolha()

print('Raspar dados dos outros pagamentos')
es.get_outros_pag()

print('Raspar dados dos outros pagamentos')
es.get_outros_pag()

print('Raspar dados dos usuários')
es.get_user()

## Obter movimento

Esta etapa é a mais demorada. Ela é responsável por baixar os documentos comprovando o envio de cada Guia.

Antes de iniciar, é preciso reabilitar os clientes inativos.

In [ ]:
es.re_enable_clients()

**IMPORTANTE:** requer que a função ```.get_clients()``` já tenha sido chamada antes.

O Exemlo abaixo usa o método ```insistir``` da biblioteca main. Este método foi criado para invocar várias vezes a função ```.get_mov()``` da classe eContabilSite, visto que o site parece parar de responder em alguns casos. Devido a ocorrência infrquente de erros e baixa reutilização deste código, optou-se por não tratar o erro e usar a força bruta (chamar novamente a função, até dar certo). Usando este método, pode-se obter em média 490 Protocolos e 490 recibos para cada hora de execução do código (imprimindo como PDF).

**Este código é persistente**. De tempos em tempos, observe o output, caso o mesmo aviso ocorra no mesmo ponto do processo, algum erro não precisto pode ter sido encontrado. Pare o processo, analise e corrija o que for preciso no código.

In [ ]:
# Caso queira pode dividir a carga em mais de um executor pyhton, nos meus testes não fez diferença, portanto pode usar 1
quant_to_split = 1 
number_bot = 0

anos = list(range(2014,2020))  #Obter dados de 2014 à 2019
meses = list(range(1,14)) #Obter dados do mês 1 ao mês 13

es = insistir(quant_to_split=quant_to_split, number_bot=number_bot, anos=anos, meses=meses)

Caso queira acompanhar o código ou realizar um **debug**, é possível executar a função ```.get_saved_clients()``` com o parâmetro ```dbg = True```, desta forma o navegador será exibido e no Console serão printados todos os erros que possam parecer.

Segue outro exemplo abaixo:

In [ ]:
df_clientes = es.get_saved_clients() #Obterm os clientes já salvos
anos = list(range(2014,2020))
meses = list(range(1,14))
dbg = True
es.get_mov(df_clientes, anos, meses, dbg)

# 3 - Correções

Mesmo com todas as medidas de precauções é possível que um ou outro erro passe desapercebido. Os comandos abaixo cruzam a relação de arquivos baixados presentes na pasta com os impostos registrados no banco de dados. Caso haja discrepância é possível corrigir utilizando algumas ações.

## a) Remover aruivos duplicados

Esta rotina localiza e exclui arquivos que foram baixados mais de uma vez. Isso pode ocorrer caso a conexão ou a automação falhe durante a captura de da competência de um cliente. Os itens já baixados não são ignorados e por segurança são baixados novamente.

In [ ]:
dk = AlwaysDoubleCheck()
dk.remove_duplicates_files()

## b) Encontrar inconsistências

Essa rotina fazr o cruzamento dos aruqivos baixados com o banco de dados. E salva no banco de dados uma relação de erros na tabela ```Erros```. Idealmente deve-se corrigir os erros de que tenham dados na coluna ```company``` antes de iniciar. Estes são arquivos que foram baixados mas não estão no banco de dados.

In [ ]:
dk.update_errors()

## c) Reprocessar Ausentes

Após corritir os erros que tenham dados na coluna ```company``` podemos focar nos erros onde evistem dados na coluna ```companyb```. Este são tributos localizados no site, mas que não foram baixados. Para reprocessa-los pode-se executar o seguinte comando:

In [ ]:
# Reprocessar
dk.rerun()

# Remover possíveis duplicatas reincluídas
dk.remove_duplicates_files()

# Revalidar
dk.update_errors()

Após essa etapa, pode-se recomeçar a análise. Estes passos podem ser repetidos até que não haja mais nenhum erro presente na tabela ```Erros```.

Se tudo estiver correto, é possível desabilitar os clientes reabilitados.

In [ ]:
es.re_disable_clients()

# 4 - Zip

Ao final do processo é possível agrupar as guias de cada cliente em um único arquivo Zip, facilitando sua sincronização com o OneDrive.

In [ ]:
# Create a folder for each client
dk.group_file_by_client()

# Create one Zip for each folder
dk.zip()